# Фреймворк PyTorch для разработки искусственных нейронных сетей

## Урок 3. Dataset, Dataloader, BatchNorm, Dropout, Оптимизация

### Практическое задание

Будем практиковаться на датасете недвижимости (sklearn.datasets.fetch_california_housing)

Ваша задача:
1. Создать Dataset для загрузки данных
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

### Решение

#### Импорт библиотек

In [2]:
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_california_housing

from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset 
from torch.utils.data import DataLoader 

from torch import nn

from torch.optim import SGD
from torch.optim import RMSprop
from torch.optim import Adam

#### 1. Создание Dataset для загрузки данных

In [11]:
# Загрузка датасета.
X, y = fetch_california_housing(return_X_y=True)

# Проверка: вывод размерностей признаков и целевой переменной.
X.shape, y.shape

((20640, 8), (20640,))

In [13]:
# Разбиение данных на обучающую и тестовую выборки.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

# Проверка: вывод размерностей.
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((15480, 8), (5160, 8), (15480,), (5160,))

In [23]:
# Класс для представления данных.
class MyDataset(Dataset):
    def __init__(self, X, y=None, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        if self.transform is not None and self.y is not None:
            return self.transform(self.X[index]), self.y[index]
        elif self.transform is not None and self.y is None:
            return self.transform(self.X[index])
        elif self.transform is None and self.y is not None:
            return self.X[index], self.y[index]
        elif self.transform is None and self.y is None:
            return self.X[index]

In [40]:
# Представление обучающей выборки в виде класса.
dataset_train = MyDataset(X_train, y_train)

# Проверка: вывод длины.
print(f'Длина:\n\t{dataset_train.__len__()}')

# Проверка: получение первого объекта.
print(f'\nПервый элемент:\n\t{dataset_train.__getitem__(0)}')

Длина:
	15480

Первый элемент:
	(array([ 3.51740000e+00,  3.60000000e+01,  4.54794521e+00,  1.09436834e+00,
        1.35700000e+03,  2.06544901e+00,  3.42100000e+01, -1.18230000e+02]), 2.68)


In [41]:
# Представление тестовой выборки в виде класса.
dataset_test = MyDataset(X_test, y_test)

# Проверка: вывод размерности.
print(f'Длина:\n\t{dataset_test.__len__()}')

# Проверка: получение первого объекта.
print(f'\nПервый элемент:\n\t{dataset_test.__getitem__(0)}')

Длина:
	5160

Первый элемент:
	(array([   4.1528    ,   15.        ,    6.19327731,    0.98739496,
        768.        ,    3.22689076,   35.34      , -119.08      ]), 1.301)


#### 2. Оборачивание данных в DataLoader

In [42]:
# Загрузчик данных обучающей выборки.
dataloader_train = DataLoader(dataset_train, batch_size=128, shuffle=True)

# Проверка загрузчика.
for batch, target in dataloader_train:
    print(f'Размерность батча признаков:\n\t{batch.shape}')
    print(f'\nПервый элемент батча признаков:\n\t{batch[0]}')
    print(f'\nРазмерность батча таргетов:\n\t{target.shape}')
    print(f'\nПервый элемент батча таргетов:\n\t{target[0]}')
    break

Размерность батча признаков:
	torch.Size([128, 8])

Первый элемент батча признаков:
	tensor([   4.9643,   37.0000,    7.1064,    1.1489,   98.0000,    2.0851,
          37.8500, -122.2400], dtype=torch.float64)

Размерность батча таргетов:
	torch.Size([128])

Первый элемент батча таргетов:
	3.35


In [43]:
# Загрузчик данных тестовой выборки.
dataloader_test = DataLoader(dataset_test, batch_size=128, shuffle=True)

# Проверка загрузчика.
for batch, target in dataloader_test:
    print(f'Размерность батча признаков:\n\t{batch.shape}')
    print(f'\nПервый элемент батча признаков:\n\t{batch[0]}')
    print(f'\nРазмерность батча таргетов:\n\t{target.shape}')
    print(f'\nПервый элемент батча таргетов:\n\t{target[0]}')
    break

Размерность батча признаков:
	torch.Size([128, 8])

Первый элемент батча признаков:
	tensor([   3.0673,   38.0000,    4.8810,    1.1161,  711.0000,    2.0142,
          37.7300, -122.1400], dtype=torch.float64)

Размерность батча таргетов:
	torch.Size([128])

Первый элемент батча таргетов:
	2.184


#### 3. Архитектура нейросети

In [44]:
# Инициализация класса нейросети:
# - сеть состоит из двух блоков с постепенным понижением размерности;
# - в каждом блоке используется функция активации "LeakyReLU", устойчивая к затуханию градиента;
# - в конце каждого блока используется слой нормализации "BatchNorm1d" для повышения эффективности следующих слоёв;
# - также в конце каждого блока используется слой "Dropout" для нивелирования эффекта переобучения;
# - на выходе используется линейный слой без функции активации для прогноза непрерывной величины.
class MyNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        
        self.fc_1 = nn.Linear(in_features=input_dim, out_features=input_dim*10)
        self.ac_1 = nn.LeakyReLU(negative_slope=0.1)
        self.fc_2 = nn.Linear(in_features=input_dim*10, out_features=input_dim*8)
        self.ac_2 = nn.LeakyReLU(negative_slope=0.1)
        self.bn_1 = nn.BatchNorm1d(num_features=input_dim*8)
        self.do_1 = nn.Dropout(0.25)
        
        self.fc_3 = nn.Linear(in_features=input_dim*8, out_features=input_dim*6)
        self.ac_3 = nn.LeakyReLU(negative_slope=0.1)
        self.fc_4 = nn.Linear(in_features=input_dim*6, out_features=input_dim*4)
        self.ac_4 = nn.LeakyReLU(negative_slope=0.1)
        self.bn_2 = nn.BatchNorm1d(num_features=input_dim*4)
        self.do_2 = nn.Dropout(0.25)
        
        self.fc_5 = nn.Linear(in_features=input_dim*4, out_features=1)
    
    
    def forward(self, x):
        x = self.fc_1(x)
        x = self.ac_1(x)
        x = self.fc_2(x)
        x = self.ac_2(x)
        x = self.bn_1(x)
        x = self.do_1(x)
        
        x = self.fc_3(x)
        x = self.ac_3(x)
        x = self.fc_4(x)
        x = self.ac_4(x)
        x = self.bn_2(x)
        x = self.do_2(x)
        
        x = self.fc_5(x)
        
        return x

#### 4. Сравнение сходимости различных методов оптимизации

In [123]:
# Функция для обучения и валидации модели.
def func_train(model, optimizer, epochs=1):
    criterion = nn.MSELoss()
    
    for epoch in range(epochs):
        
        for i, data_train in enumerate(dataloader_train):

            optimizer.zero_grad()

            pred_train = model(data_train[0].float())
            loss_train = criterion(pred_train, data_train[1].float())
            loss_train.backward()
            optimizer.step()

            if i % 50 == 0:
                model.eval()
                
                for _, data_test in enumerate(dataloader_test):
                    pred_test = model(data_test[0].float())
                    loss_test = criterion(pred_test, data_test[1].float())
                
                model.train()            
            
                print(
                    f'Epoch {epoch + 1}/{epochs}\t\t' \
                    f'train loss {round(loss_train.item(), 2)}\t\t' \
                    f'train loss {round(loss_test.item(), 2)}.'
                )

    print('Training is finished!')

##### Метод оптимизации SGD

In [144]:
model_sgd = MyNet(X.shape[1])

optimizer_sgd = SGD(model_sgd.parameters(), lr=0.001)

In [145]:
%%time

func_train(model_sgd, optimizer_sgd, epochs=5)

Epoch 1/5		train loss 5.89		train loss 4.4.
Epoch 1/5		train loss 5.37		train loss 3.95.
Epoch 1/5		train loss 3.62		train loss 4.39.
Epoch 2/5		train loss 3.09		train loss 4.76.
Epoch 2/5		train loss 3.13		train loss 2.85.
Epoch 2/5		train loss 3.87		train loss 2.66.
Epoch 3/5		train loss 2.81		train loss 2.13.
Epoch 3/5		train loss 2.57		train loss 1.92.
Epoch 3/5		train loss 2.4		train loss 1.08.
Epoch 4/5		train loss 2.33		train loss 3.24.
Epoch 4/5		train loss 1.87		train loss 2.48.
Epoch 4/5		train loss 1.83		train loss 1.73.
Epoch 5/5		train loss 2.47		train loss 1.01.
Epoch 5/5		train loss 1.29		train loss 2.55.
Epoch 5/5		train loss 1.55		train loss 2.47.
Training is finished!
Wall time: 1.54 s


##### Метод оптимизации RMSProp

In [146]:
model_RMSprop = MyNet(X.shape[1])

optimizer_RMSprop = RMSprop(model_RMSprop.parameters(), lr=0.001)

In [147]:
%%time

func_train(model_RMSprop, optimizer_RMSprop, epochs=5)

Epoch 1/5		train loss 4.66		train loss 30.53.
Epoch 1/5		train loss 2.82		train loss 2.95.
Epoch 1/5		train loss 2.59		train loss 1.93.
Epoch 2/5		train loss 2.21		train loss 2.21.
Epoch 2/5		train loss 1.45		train loss 1.15.
Epoch 2/5		train loss 1.65		train loss 1.23.
Epoch 3/5		train loss 1.79		train loss 1.28.
Epoch 3/5		train loss 1.59		train loss 1.64.
Epoch 3/5		train loss 1.41		train loss 1.18.
Epoch 4/5		train loss 1.5		train loss 1.06.
Epoch 4/5		train loss 1.86		train loss 1.29.
Epoch 4/5		train loss 1.45		train loss 1.39.
Epoch 5/5		train loss 1.61		train loss 1.87.
Epoch 5/5		train loss 1.45		train loss 0.8.
Epoch 5/5		train loss 2.06		train loss 0.92.
Training is finished!
Wall time: 1.69 s


##### Метод оптимизации Adam

In [148]:
model_Adam = MyNet(X.shape[1])

optimizer_Adam = Adam(model_Adam.parameters(), lr=0.001)

In [149]:
%%time

func_train(model_Adam, optimizer_Adam, epochs=5)

Epoch 1/5		train loss 7.46		train loss 7.38.
Epoch 1/5		train loss 5.49		train loss 6.04.
Epoch 1/5		train loss 4.96		train loss 4.84.
Epoch 2/5		train loss 3.59		train loss 5.01.
Epoch 2/5		train loss 2.87		train loss 2.39.
Epoch 2/5		train loss 2.38		train loss 1.59.
Epoch 3/5		train loss 2.21		train loss 1.27.
Epoch 3/5		train loss 2.8		train loss 1.65.
Epoch 3/5		train loss 1.36		train loss 1.14.
Epoch 4/5		train loss 1.71		train loss 1.22.
Epoch 4/5		train loss 1.68		train loss 1.5.
Epoch 4/5		train loss 1.56		train loss 1.62.
Epoch 5/5		train loss 1.69		train loss 0.98.
Epoch 5/5		train loss 1.24		train loss 1.91.
Epoch 5/5		train loss 1.77		train loss 1.23.
Training is finished!
Wall time: 1.89 s


##### Выводы

В рамках решения данной задачи различные методы оптимизации демонстрировали различное качество метрики <code>MSE</code> от запуска к запуску. Однако оптимизатор <code>Adam</code> чаще остальных отличался в лучшую сторону меньшим переобучением.

Также стоит отметить, что <code>Adam</code> в среднем работает чуть дольше, чем <code>SGD</code> и <code>RMSprop</code>, что может быть объяснено сложностью алгоритма.